In [74]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.dummy import DummyClassifier
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler

overview of this approach is I want to use log reg to get probability functions for each class (ie odds that data point is that class). Since 1/(1+e^{-t}) is increasing and I can get back the coeffs for determining t, we'll use whatever t value is the highest to pick the class.

In [75]:
df = pd.read_csv('EpcProject2.csv')
#Get rid of hexidecimals
df = df.apply(lambda x: [int(i, 16) for i in x] if x.name in '1236' else x)
df.replace({'A':10, 'B':11, 'C':12, 'D':13, 'E':14, 'F':15, 'G':16}, inplace=True)

df['Checkers'] = df['1'] + df['2'] + df['3'] + df['4'] + df['5'] + df['6']


df_high = df.loc[df['Description']=='High']
df_med = df.loc[df['Description']=='Medium']
df_low = df.loc[df['Description']=='Low']
df_rol = df.loc[df['Description']=='Rollish']


df['Men off']= df['Men off'].astype(int)
df['Odd']= (df['Checkers']%2==1).astype(int)
df['CG'] = (df['PC']/df['Checkers'] - df['Odd']/2)
df['CG_N']= (df['CG']/df['CG'].max())*2
df['gap1']= (df['1']==0).astype(int)
df['gap2']= (df['2']==0).astype(int)
df['gap3']= (df['3']==0).astype(int)
df['gap4']= (df['4']==0).astype(int)
df['gap5']= (df['5']==0).astype(int)
df['gap6']= (df['6']==0).astype(int)


df['tower1'] = (df['1'] >= 5).astype(int)
df['tower2'] = (df['2'] >= 5).astype(int)
df['tower3'] = (df['3'] >= 5).astype(int)
df['tower4'] = (df['4'] >= 5).astype(int)
df['tower5'] = (df['5'] >= 5).astype(int)
df['tower6'] = (df['6'] >= 5).astype(int)
df['towers'] = df['tower1'] + df['tower2'] + df['tower3'] + df['tower4'] + df['tower5'] + df['tower6']

df['acedeuce'] = (df['1']+df['2'])
df['highlow'] = df[['6', '5', '4', '3', '2', '1']].max(axis=1) - df[['6', '5', '4', '3', '2', '1']].min(axis=1)

df['High'] = df['Description'] == 'High'
df['Medium'] = df['Description'] == 'Medium'
df['Low'] = df['Description'] == 'Low'
df['Rollish'] = df['Description'] == 'Rollish'


# df = df.loc[df['Checkers'] == 6]

In [76]:
columns = [
'Odd',
'gap1',
'gap2',
'gap3',
'gap4',
'gap5',
'gap6',
'highlow', 
'Checkers',
'1',
'2',
'3',
'4',
'5',
'6', 
'PC']
std_scalar = StandardScaler()
df_scaled = df.copy()
df_scaled[columns] = std_scalar.fit_transform(df_scaled[columns])

Looking for the params to use we're gonna scale the data set and check

In [66]:
X = df_scaled[columns]
y_h = df_scaled['High']

In [67]:
model_h = LogisticRegression(penalty='l1', solver='liblinear', fit_intercept=False)
model_h.fit(X,y_h)
model_h.score(X,y_h)
print(model_h.coef_.tolist()[0])

attributes = [(-10000,0), (-10000,0)]
for i, coef in enumerate(model_h.coef_.tolist()[0]):
    if abs(coef) >= max(attributes)[0]:
        attributes.remove(min(attributes))
        attributes.append((coef, i))
attributes

[-0.018271762311709603, 0.38278351385824244, 0.19410986301373817, -0.03920361422904829, -0.07283608036083083, 0.18434412227159475, 0.14794147569509378, 0.2746864536689803, -0.14907138944061055, -0.10483658396653275, -0.19057776895188935, -0.24100782231843712, 0.0, 0.5019781948262501, 0.314155430638792, 0.0]


[(0.38278351385824244, 1), (0.5019781948262501, 13)]

In [68]:
columns[13]

'5'

Starting with High

In [82]:
X = df[['acedeuce', '3']]
y_h = df['High']

In [83]:
model_h = LogisticRegression(penalty='l1', solver='liblinear')
model_h.fit(X,y_h)
model_h.score(X,y_h)
print(model_h.coef_)
print(model_h.intercept_)
print(model_h.score(X,y_h))
# coeffs = model_h.coef_.tolist()[0]
# for i,ele in enumerate(coeffs):
#     coeffs[i] = abs(ele)
# print(coeffs)
# print(coeffs.index(max(coeffs)))
# columns[19]

[[-2.17605778 -0.99015307]]
[3.1934854]
0.9359005605654399


Medium next

In [85]:
X = df_scaled[columns]
y_m = df_scaled['Medium']

In [86]:
model_m = LogisticRegression(penalty='l1', solver='liblinear')
model_m.fit(X,y_m)
print(model_m.coef_)
print(model_m.score(X,y_m))

[[-0.45847717 -0.73342765 -0.23704084  0.20588325  0.21501029 -0.13082777
  -0.03680957  0.45680618  0.05033837  0.01605361 -0.01343911 -0.25287093
  -0.23080992  0.         -0.91506137  0.10565572 -0.07319697 -1.11521008
   0.          0.22597834  0.17072743 -0.04750196  0.          0.
   0.00451842]]
0.7354322490867977


In [127]:
y_l = df['Low']

In [130]:
X = df[['CG', 'acedeuce', 'Odd']]
model_l = LogisticRegression(penalty='l1', solver='liblinear')
model_l.fit(X,y_l)
model_l.score(X,y_l)

0.785701861193251

Scaling the dataset to lmk whats actually useful

In [77]:
X = df_scaled[columns]
y_r = df_scaled['Rollish']

In [78]:
model_r = LogisticRegression(penalty='l1', solver='liblinear')
model_r.fit(X,y_r)
model_r.score(X,y_r)
print(model_r.coef_)
columns[14]
columns[-2]

[[ 2.42905658 -1.14092589 -0.76683569 -0.86653628 -0.95030125 -0.4753547
  -0.06133433 -2.24630132  0.          3.46826529  2.2332344   0.59538461
  -0.60758646 -2.03960065 -3.80347393 -2.04456007]]


'6'

In [79]:
attributes = [(-10000,0), (-10000,0)]
for i, coef in enumerate(model_r.coef_.tolist()[0]):
    if abs(coef) >= max(attributes)[0]:
        attributes.remove(min(attributes))
        attributes.append((coef, i))
attributes

[(3.4682652899657556, 9), (-3.8034739318002875, 14)]

In [81]:
columns[14]

'6'

In [30]:
X = df[['acedeuce', 'PC']]
y_r = df['Rollish']

In [32]:
model_r = LogisticRegression(penalty='l1', solver='liblinear')
model_r.fit(X,y_r)
print(model_r.score(X,y_r))
print(model_r.coef_)
print(model_r.intercept_)

0.8771960340928857
[[ 0.63148222 -0.11473601]]
[-1.42629729]


In [97]:
# no idea if a more compact form will be more useful for testing so im adding it
X = df[['PC', 'Men off', 'Odd']]
y_h = df['High']
y_m = df['Medium']
y_l = df['Low']
y_r= df['Rollish']

model_h = LogisticRegression(penalty='l2', solver='liblinear')
model_h.fit(X,y_h)

model_m = LogisticRegression(penalty='l2', solver='liblinear')
model_m.fit(X,y_m)

model_l = LogisticRegression(penalty='l2', solver='liblinear')
model_l.fit(X,y_l)

model_r = LogisticRegression(penalty='l2', solver='liblinear')
model_r.fit(X,y_r)

LogisticRegression(solver='liblinear')

In [98]:
class Model:
    def __init__(self, models:list, round_deg = 64):
        self.models = models
        self.round_deg = round_deg
        self.decisions = []
        for i, model in enumerate(self.models):
            decision = model.decision_function(X)
            self.decisions.append((decision,i)) #to make sure we know what model is doing the scoring
    
    def predict(self, X):
        y_pred = []
        for i in range(len((X))):
            max_score = (-10000000, 0)
            for decision in self.decisions:
                score = (decision[0][i], decision[1])
                max_score = max(max_score, score)
            if max_score[1] == 0:
                y_pred.append('High')
            if max_score[1] == 1:
                y_pred.append('Medium')
            if max_score[1] == 2:
                y_pred.append('Low')
            if max_score[1] == 3:
                y_pred.append('Rollish')
        return y_pred
    

In [99]:
model = Model([model_h, model_m, model_l, model_r])
y_pred = model.predict(X)

In [100]:
accuracy_score(df['Description'],y_pred)

0.6742042094277265